In [16]:
import cv2
import mediapipe as mp
import numpy as np
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Initialize Pycaw for volume control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Function to calculate the distance between two points
def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    success, image = cap.read()
    if not success:
        break

    # Convert the BGR image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    result = hands.process(image_rgb)

    # Control volume based on hand gesture
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

            # Convert normalized coordinates to pixel values
            thumb_tip_coords = (int(thumb_tip.x * image.shape[1]), int(thumb_tip.y * image.shape[0]))
            index_finger_tip_coords = (int(index_finger_tip.x * image.shape[1]), int(index_finger_tip.y * image.shape[0]))

            # Calculate the distance between thumb tip and index finger tip
            distance = calculate_distance(thumb_tip_coords, index_finger_tip_coords)

            # Map the distance to a volume range
            min_distance = 20  # Adjust based on your hand size and webcam distance
            max_distance = 200  
            volume_range = volume.GetVolumeRange()
            min_volume, max_volume = volume_range[0], volume_range[1]

            normalized_distance = np.clip((distance - min_distance) / (max_distance - min_distance), 0.0, 1.0)
            new_volume = min_volume + (max_volume - min_volume) * normalized_distance

            volume.SetMasterVolumeLevel(new_volume, None)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()


c:\Users\draar\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 